### SETUP

In [1]:
### 1. Mount Google Drive ###

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
### 2. Prepare data ###

!scp '/content/gdrive/My Drive/YOLOTRAIN2/data.zip' '/content/data.zip'

!unzip '/content/data.zip' -d '/content/'

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/labels/train/VID_20250306_151647_mp4-0973_jpg.rf.1363cb420d8c01265c905a2017611865.txt  
  inflating: /content/data/labels/train/VID_20250306_151647_mp4-0973_jpg.rf.84c29680354bd22d201f121ca3af3f6c.txt  
  inflating: /content/data/labels/train/VID_20250306_151647_mp4-0973_jpg.rf.9ca17fb7c4c64eb22624027abbc03aac.txt  
  inflating: /content/data/labels/train/VID_20250306_151647_mp4-0974_jpg.rf.bd4ff0ee0847d6901f5efe681ea581ec.txt  
  inflating: /content/data/labels/train/VID_20250306_151647_mp4-0974_jpg.rf.e32b2e1413dfcecc161985140e78ade5.txt  
  inflating: /content/data/labels/train/VID_20250306_151647_mp4-0975_jpg.rf.474be29025948a87a1e70b8f44b50279.txt  
  inflating: /content/data/labels/train/VID_20250306_151647_mp4-0975_jpg.rf.9b7897c928964036f49e6dcf5c5fa07e.txt  
  inflating: /content/data/labels/train/VID_20250306_151647_mp4-0977_jpg.rf.515a1e653da62e538cbad78c77c119a6.txt  
  inflating: /content/data/la

In [3]:
### 3. Install packages ###

!git clone https://github.com/allansdefreitas/yolov10.git
!cd yolov10 && pip install

Cloning into 'yolov10'...
remote: Enumerating objects: 20338, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 20338 (delta 4), reused 4 (delta 4), pack-reused 20330 (from 2)
Receiving objects: 100% (20338/20338), 11.11 MiB | 12.38 MiB/s, done.
Resolving deltas: 100% (14354/14354), done.
ERROR: You must give at least one requirement to install (see "pip help install")


In [4]:
import os

!pip install ultralytics --upgrade --pre

# Define the folder name you used during training
run_name = "SGD_S2"  # Rename after every hyperparameter tuning session
project_path = f"/content/gdrive/MyDrive/YOLOTRAIN2/{run_name}"

import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip show torch ultralytics
!yolo version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [5]:
### DEVICE CHECK
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


CUDA available: True
Device name: Tesla T4


### Model Training

In [6]:
### 4.0 Import ultralytics and train
import os

!pip install ultralytics --upgrade --pre

from ultralytics import YOLO

# Define the config.yaml path
config_path = '/content/gdrive/My Drive/YOLOTRAIN2/config.yaml'

# Load YOLOv10n model properly
model = YOLO("jameslahm/yolov10n")  # load pre-trained YOLOv10n

# Define the folder name you used during training
os.makedirs(project_path, exist_ok=True)

# Train the model
model.train(
    data=config_path,
    epochs=100,
    batch= 64,
    optimizer='SGD',
    lr0=0.01,
    lrf=0.01,
    cos_lr=True,
    momentum=0.9,
    weight_decay=0.001,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    close_mosaic=10,
    patience=20,
    resume=False,
    project=project_path,
    name="train",
    device=0
    )


100%|██████████| 5.59M/5.59M [00:00<00:00, 258MB/s]


Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/gdrive/My Drive/YOLOTRAIN2/config.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=jameslahm/yolov10n.pt, momentum=0.9, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=20, perspective=0.0, plots=True

100%|██████████| 755k/755k [00:00<00:00, 123MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1      9856  ultralytics.nn.modules.block.SCDown          [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1     36096  ultralytics

YOLOv10n summary: 223 layers, 2,707,430 parameters, 2,707,414 gradients, 8.4 GFLOPs

Transferred 493/595 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

                   all       3250      13716      0.969      0.981      0.989      0.763



100 epochs completed in 6.720 hours.
Optimizer stripped from /content/gdrive/MyDrive/YOLOTRAIN2/SGD_S2/train/weights/last.pt, 5.8MB
Optimizer stripped from /content/gdrive/MyDrive/YOLOTRAIN2/SGD_S2/train/weights/best.pt, 5.8MB

Validating /content/gdrive/MyDrive/YOLOTRAIN2/SGD_S2/train/weights/best.pt...
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10n summary (fused): 102 layers, 2,265,363 parameters, 0 gradients, 6.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:36<00:00,  1.39s/it]


                   all       3250      13716      0.969      0.981      0.989      0.763
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /content/gdrive/MyDrive/YOLOTRAIN2/SGD_S2/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f9e2a523990>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [7]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip show torch ultralytics
!yolo version
!yolo cfg

Name: torch
Version: 2.6.0+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchdata, torchvision, ultralytics, ultralytics-thop
---
Name: ultralytics
Version: 8.3.156
Summary: Ultralytics YOLO 🚀 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: https://ultralytics.com
Author: 
Autho

### Continue from Disconnection

In [ ]:
### 4.1. Continue training from disconnected session

from ultralytics import YOLO

# Load the last checkpoint from Google Drive
checkpoint_path = f"{project_path}/train/weights/last.pt"
model = YOLO(checkpoint_path)

# Resume training from last epoch
model.train(
    resume=True,
    project= project_path,
    name= "train"
)